# DSCI 525 Milestone 1 - Group 21

Github Repo: https://github.com/UBC-MDS/DSCI_525_group21

In [29]:
import io
import os
import json
import glob
#import intake
import requests
import numpy as np
import pandas as pd
#import xarray as xr
from urllib.request import urlretrieve
#import proplot as pplot
#from joblib import Parallel, delayed
#import warnings
#warnings.filterwarnings("ignore")  # ignore some annoying matplotlib warnings
from memory_profiler import memory_usage
import zipfile

In [30]:
%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# Download the data from figshare to local computer

In [31]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [32]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

As you can see, there are 5 files. The one we want is the first one, "data.zip":

In [33]:
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [34]:
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

# Combining CSVs

In [35]:
### just listing to get an idea how individual file looks like 
use_cols = ["time",'lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)']
df = pd.read_csv("./figshare/ACCESS-CM2_daily_rainfall_NSW.csv", usecols=use_cols)
df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [47]:
%%time
%memit

# Shows time that regular python takes to merge file
# Join all data together
import pandas as pd
use_cols = ["time",'lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)']

files = glob.glob('./figshare/*.csv')

df_all = None

for file in files:
    
    filename = os.path.basename(file)
    
    if '_daily_rainfall_NSW.csv' in filename:
        print(f"Processing the file {filename}")
        model = filename.split('_daily_rainfall_NSW.csv')[0]

        df = pd.read_csv(file, usecols=use_cols, index_col=0)
        df['model'] = model    

        if df_all is None:
            df_all = df
        else:
            df_all = df_all.append(df)


peak memory: 1593.48 MiB, increment: 0.00 MiB
Processing the file, AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
Processing the file, MIROC6_daily_rainfall_NSW.csv
Processing the file, FGOALS-f3-L_daily_rainfall_NSW.csv
Processing the file, NorESM2-MM_daily_rainfall_NSW.csv
Processing the file, observed_daily_rainfall_SYD.csv
Processing the file, EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
Processing the file, CanESM5_daily_rainfall_NSW.csv
Processing the file, NESM3_daily_rainfall_NSW.csv
Processing the file, GFDL-ESM4_daily_rainfall_NSW.csv
Processing the file, MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
Processing the file, ACCESS-CM2_daily_rainfall_NSW.csv
Processing the file, FGOALS-g3_daily_rainfall_NSW.csv
Processing the file, INM-CM5-0_daily_rainfall_NSW.csv
Processing the file, MRI-ESM2-0_daily_rainfall_NSW.csv
Processing the file, MPI-ESM1-2-HR_daily_rainfall_NSW.csv
Processing the file, SAM0-UNICON_daily_rainfall_NSW.csv
Processing the file, NorESM2-LM_daily_rainfall_NSW.csv
Processing the f

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,3.129635e-02,AWI-ESM-1-1-LR
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,1.083881e-13,AWI-ESM-1-1-LR
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,1.056313e-13,AWI-ESM-1-1-LR
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,1.080510e-13,AWI-ESM-1-1-LR
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,9.914916e-14,AWI-ESM-1-1-LR
...,...,...,...,...,...,...
2014-12-27 12:00:00,-30.000000,-28.500000,153.0000,155.0000,3.602377e-01,INM-CM4-8
2014-12-28 12:00:00,-30.000000,-28.500000,153.0000,155.0000,5.941230e+00,INM-CM4-8
2014-12-29 12:00:00,-30.000000,-28.500000,153.0000,155.0000,1.054071e+01,INM-CM4-8


In [48]:
df_all.to_csv('./figshare/combined_data.csv')

In [49]:
%%sh
du -sh figshare/combined_data.csv

5.6G	figshare/combined_data.csv


## Observations

JUFU - Laptop with 32GB RAM and i5 processor takes:
* 1min 7s CPU time
* Peak memory 1593MB

# Load combined CSV in memory and perform simple EDA

**Peak memory**: peak memory usage of your system (including memory usage of other processes) during the program runtime.

**Increment**: the increment in memory usage relative to the memory usage just before the program is run 

## Loading in chunks

In [53]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshare/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 6511.22 MiB, increment: 2100.35 MiB
CPU times: user 55.6 s, sys: 7.37 s, total: 1min 2s
Wall time: 1min 3s


In [51]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshare/combined_data.csv", chunksize=1_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 4595.90 MiB, increment: 82.60 MiB
CPU times: user 56.9 s, sys: 1.84 s, total: 58.7 s
Wall time: 58.8 s


In [52]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshare/combined_data.csv", chunksize=500_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 4424.06 MiB, increment: 0.00 MiB
CPU times: user 53 s, sys: 910 ms, total: 53.9 s
Wall time: 54 s


## Observations

The larger the chunk size, the more peak_memory that is needed to handle it. 
The smaller the chunk size, the less CPU time is needed to go through all chunks.

The following are the results comparing chunksize to the increment of memory needed to handle the chunksize.

- 500_000 chunks: peak memory: 4424.06 MiB, increment: 0.00 MiB, CPU_time: total: 1min 2s
- 1_000_000 chunks: peak memory: 4595.90 MiB, increment: 82.60 MiB, CPU_time: 58.7s
- 10_000_000 chunks: peak memory: 6511.22 MiB, increment: 2100.35 MiB, CPU_time: 53.9s


In [54]:
import pandas as pd
## install the packages https://arrow.apache.org/docs/python/install.html
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
## How to install put instructions https://anaconda.org/conda-forge/rpy2
import rpy2.rinterface
# install this https://pypi.org/project/rpy2-arrow/#description  pip install rpy2-arrow
# have to install this as well conda install -c conda-forge r-arrow 
import rpy2_arrow.pyarrow_rarrow as pyra
### instruction
import pyarrow.feather as feather

In [55]:
%%R
#just seeing if its available
library("arrow")
library("dplyr")

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Perform simple EDA in R

In [57]:
%%time
%%memit
## read more on the datasets here  https://arrow.apache.org/docs/python/dataset.html
dataset = ds.dataset("figshare/combined_data.csv", format="csv")
## this is of arrow table format
table = dataset.to_table()

peak memory: 8673.10 MiB, increment: 4079.01 MiB
CPU times: user 18.5 s, sys: 2.05 s, total: 20.6 s
Wall time: 19.6 s


In [58]:
%%time
# experiment in writing in feather format 
feather.write_feather(table, 'figshare/figshare.feather')

CPU times: user 2.83 s, sys: 980 ms, total: 3.81 s
Wall time: 2.01 s


In [59]:
%%time
%%R
### her we are showing how much time it took to read a feather file what we wrote in python
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figshare/figshare.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "tbl_df"     "tbl"        "data.frame"
# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 5.342926 secs
CPU times: user 7.06 s, sys: 3.88 s, total: 10.9 s
Wall time: 5.4 s


## Why I chose Feather?

I chose the feather data format - which was designed to improve data interoperability (ex. language agnostic - R and Python) while dealing with columnar tabular data. I also wanted the fastest load and save times. Feather is a fast, lightweight, and easy-to-use binary file format. Feather also doesn't use compression internally and works best with SSD drives (which my laptop has).

However, if I needed to store this for long term storage, I would not use Feather because the file format is relatively new and can change. Feather does not guarantee if the format will stay the same between versions.

